In [1]:
!mkdir -p /scratch/sagarsj42/torch-cache
!mkdir -p /scratch/sagarsj42/transformers
import os
os.chdir('/scratch/sagarsj42')
os.environ['TORCH_HOME'] = '/scratch/sagarsj42/torch-cache'
os.environ['TRANSFORMERS_CACHE'] = '/scratch/sagarsj42/transformers'

In [2]:
from functools import partial

import pandas as pd
import numpy as np

# import torch
# from torch import nn
# from torch.nn import functional as F
# from torch.utils.data import Dataset, DataLoader
# from torch.optim import AdamW

# import pytorch_lightning as pl
# from pytorch_lightning.callbacks import ModelCheckpoint
# import torchmetrics

from sklearn.linear_model import LinearRegression

# from transformers import XLMRobertaTokenizer, XLMRobertaModel

In [3]:
!scp sagarsj42@ada:/share1/sagarsj42/semeval8-train-sss.csv .
!scp sagarsj42@ada:/share1/sagarsj42/semeval8-dev.csv .
!scp sagarsj42@ada:/share1/sagarsj42/semeval-2022-task-8-eval-df.csv .

semeval8-train-sss.csv                        100%   25MB  24.5MB/s   00:00    
semeval8-dev.csv                              100% 2825KB   2.8MB/s   00:00    
semeval-2022-task-8-eval-df.csv               100%   27MB  27.4MB/s   00:00    


In [4]:
train1 = 'mlns-xlmr-regressor-train-results.csv'
dev1 = 'mlns-xlmr-regressor-dev-results.csv'
test1 = 'mlns-xlmr-regressor-test-results.csv'

In [5]:
train2 = 'mlns-ner-title-distilbert-regressor-train-results.csv'
dev2 =  'mlns-ner-title-distilbert-regressor-dev-results.csv'
test2 = 'mlns-ner-title-distilbert-regressor-test-results.csv'

In [6]:
train1_df = pd.read_csv(train1)
dev1_df = pd.read_csv(dev1)
test1_df = pd.read_csv(test1)

train1_df.shape, dev1_df.shape, test1_df.shape

((4467, 3), (497, 3), (4953, 3))

In [7]:
train2_df = pd.read_csv(train2)
dev2_df = pd.read_csv(dev2)
test2_df = pd.read_csv(test2)

train2_df.shape, dev2_df.shape, test2_df.shape

((4467, 3), (497, 3), (4953, 3))

In [8]:
train1_df.head()

,Unnamed: 0,pair_id,Overall
0,0,1484208071_1484059708,1.828174
1,1,1493864787_1541999376,1.846863
2,2,1484763358_1484635842,3.457969
3,3,1483805573_1484265728,3.549678
4,4,1605429954_1591825952,3.532647


In [11]:
train2_df.head()

,Unnamed: 0,pair_id,Overall
0,0,1484208071_1484059708,1.816404
1,1,1493864787_1541999376,2.905895
2,2,1484763358_1484635842,2.801751
3,3,1483805573_1484265728,3.768483
4,4,1605429954_1591825952,3.664754


In [12]:
train_df = pd.read_csv('semeval8-train-sss.csv')
dev_df = pd.read_csv('semeval8-dev.csv')
test_df = pd.read_csv('semeval-2022-task-8-eval-df.csv')

train_df.shape, dev_df.shape, test_df.shape

((4467, 21), (497, 21), (4953, 21))

In [13]:
train_df.head()

,Unnamed: 0,pair_id,score,id_1,title_1,text_1,keywords_1,meta_keywords_1,tags_1,summary_1,...,meta_lang_1,id_2,title_2,text_2,keywords_2,meta_keywords_2,tags_2,summary_2,meta_description_2,meta_lang_2
0,1611,1484208071_1484059708,2.500000,1484208071,"Koalition: Hofer warnt vor ""Linksregierung""",FPÖ-Chef: 'Noch nie war eine Regierung persone...,[],[''],[],NaN,...,de,1484059708,Österreich: Neues Kabinett nimmt Form an,"Karl Nehammer wird Innenminister, Klaudia Tann...",[],[''],['Politik'],NaN,Das Kabinett der offenbar ins Haus stehenden t...,de
1,4317,1493864787_1541999376,3.000000,1493864787,WhatsApp: Diese beliebte Funktion gibt es in D...,Messenger\n\nWhatsApp-Chats ließen sich per Ba...,[],[''],[],NaN,...,de,1541999376,WhatsApp dark mode introduced: 5 WhatsApp feat...,WhatsApp Dark mode is now available for all An...,[],"['whatsapp features', 'upcoming whatsapp featu...",[],NaN,WhatsApp is testing some more feature that wil...,en
2,829,1484763358_1484635842,3.666667,1484763358,'It Warmed My Heart': Sachin Tendulkar Rings i...,"Master blaster Sachin Tendulkar, on Wednesday,...",[],"['2020', 'cricket fan', 'cricket video viral',...",[],NaN,...,NaN,1484635842,Nia Sharma Begins Her New Year with a Dip in t...,Naagin 4 star Nia Sharma is making a bold styl...,[],"['lifestyle', 'Naagin 4', 'Nia Sharma', 'pictu...",[],NaN,TV actress Nia Sharma flaunted her toned body ...,NaN
3,518,1483805573_1484265728,4.000000,1483805573,An Alabama woman texted ‘I feel in trouble’ be...,× An Alabama woman texted ‘I feel in trouble’ ...,[],"['national & world news', '']",[],NaN,...,en,1484265728,Agencies receive firm support,Wallbridge Wallbridge law firm in Timmins dona...,[],[''],[],NaN,Wallbridge Wallbridge law firm in Timmins dona...,en
4,4312,1605429954_1591825952,4.000000,1605429954,Parlament in Winterthur – Teure Ratsdebatten i...,Parlament in Winterthur – Teure Ratsdebatten i...,[],[''],[],NaN,...,de,1591825952,Coronavirus and Your Supply Chain,Sign Me Up to receive the free Logistics Brief...,[],[''],[],NaN,NaN,en


In [14]:
train_score = train_df.score.values
dev_score = dev_df.score.values
test_score = test_df.score.values

train_score.shape, dev_score.shape, test_score.shape

((4467,), (497,), (4953,))

In [15]:
train_x = np.array(list(zip(train1_df.Overall.tolist(), train2_df.Overall.tolist())))
train_x.shape

(4467, 2)

In [16]:
dev_x = np.array(list(zip(dev1_df.Overall.tolist(), dev2_df.Overall.tolist())))
dev_x.shape

(497, 2)

In [17]:
test_x = np.array(list(zip(test1_df.Overall.tolist(), test2_df.Overall.tolist())))
test_x.shape

(4953, 2)

In [18]:
model = LinearRegression(normalize=False)
model

LinearRegression(normalize=False)

In [19]:
model.fit(train_x, train_score)

/home2/sagarsj42/miniconda3/envs/sklearn/lib/python3.7/site-packages/sklearn/linear_model/_base.py:155: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2. Please leave the normalize parameter to its default value to silence this warning. The default behavior of this estimator is to not do any normalization. If normalization is needed please use sklearn.preprocessing.StandardScaler instead.
  FutureWarning,


LinearRegression(normalize=False)

In [20]:
model.score(train_x, train_score)

0.5454747029483719

In [27]:
dev_pred = np.clip(model.predict(dev_x), 1.0, 4.0)
dev_pred.shape

(497,)

In [28]:
test_pred = np.clip(model.predict(test_x), 1.0, 4.0)
test_pred.shape

(4953,)

In [29]:
test_pred.tolist()

[4.0,
 2.3405094492304297,
 3.4775601370777878,
 2.066795812212419,
 3.89905446391567,
 3.9074640751164287,
 3.339527683741325,
 2.125015827182649,
 3.9306041496459994,
 3.4742749071037746,
 3.8695683723942498,
 3.3812156458322917,
 2.039815144835961,
 4.0,
 1.9858000774197055,
 3.683978889011879,
 2.99375303564645,
 3.9013367190574835,
 3.1455316933169604,
 3.926762276630887,
 1.6586052284630863,
 2.9121292499396123,
 3.710276186536867,
 1.7349303352214465,
 4.0,
 1.7719189659017127,
 4.0,
 3.8990409807256183,
 4.0,
 3.6956681028346576,
 3.9105200409610843,
 3.8988710301015237,
 3.9003309789256395,
 1.7724832633887009,
 3.350235902902215,
 2.560796575475261,
 4.0,
 3.663083417542638,
 2.1716307671228217,
 2.2586112665621063,
 3.0281823037154068,
 3.766908033694274,
 3.9462370804532787,
 4.0,
 3.857072951680708,
 2.117650667955152,
 2.776740018040134,
 3.2860807604274527,
 3.0070440833020697,
 1.5241975441970936,
 2.551961895369547,
 3.0757751015029906,
 4.0,
 2.1912500584231793,
 3.54

In [30]:
dev_pred

array([3.68524463, 2.34779782, 1.53303789, 2.38710219, 3.54716752,
       2.1539043 , 2.91825602, 3.22839926, 2.96328967, 2.4291574 ,
       3.47989439, 3.68809697, 1.67260189, 2.29901896, 1.41759532,
       2.373906  , 2.0643278 , 1.46850894, 2.78455188, 2.30475541,
       2.27919623, 3.63616108, 3.57387756, 2.16267823, 3.16393079,
       3.03074084, 1.91456653, 1.66884124, 1.8675777 , 3.76488677,
       1.52074522, 3.64649249, 2.29969724, 3.92283002, 3.49683868,
       2.7116538 , 2.02859995, 3.16061004, 2.77232591, 3.04705933,
       2.15319199, 2.82856692, 3.3591878 , 3.49906734, 2.56744034,
       1.71363315, 1.5411728 , 1.66328341, 1.63796211, 1.97136821,
       2.32917826, 1.75080652, 2.21806124, 1.9975268 , 2.78866163,
       3.90643118, 2.83380275, 1.86244964, 3.42024646, 2.33675268,
       2.74597847, 2.16946564, 3.46942403, 2.08805798, 1.84508937,
       3.71910727, 3.96535763, 2.76981855, 2.37672617, 3.95784888,
       2.80710261, 2.38757122, 3.09255523, 3.6972437 , 1.85419

In [31]:
np.corrcoef(dev_pred, dev_score)

array([[1.        , 0.51412102],
       [0.51412102, 1.        ]])

In [33]:
pd.DataFrame(list(zip(test_df.pair_id, test_pred.tolist())), columns=['pair_id', 'Overall']).to_csv('weighted-average-distilbert-ner-xlmr-text' + 'test-results.csv')